<a href="https://www.kaggle.com/code/devrayrob/gen-ai-capstone-project-customer-support-agent?scriptVersionId=235065008" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Setup and Configuration
# This cell imports the necessary libraries and configures the Gemini API.

In [1]:
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

# Get the API key from Kaggle Secrets
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("GOOGLE_API_KEY")

# Configure Gemini API
genai.configure(api_key=api_key)

# Initialize the Gemini model with a suitable model
model = genai.GenerativeModel('models/gemini-1.5-pro-latest')

print("Gemini API configured and model initialized.")

Gemini API configured and model initialized.


# Vector Store Implementation
# This cell defines a basic `VectorStore` class to simulate storing and searching document embeddings.

In [2]:
import google.generativeai as genai
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Simulated vector store for embeddings (Day 2: Embeddings and Vector Stores)
class VectorStore:
    def __init__(self):
        self.embeddings = []
        self.documents = []

    def add_document(self, doc):
        embedding = genai.embed_content(
            model='models/embedding-001',
            content=doc,
            task_type='RETRIEVAL_DOCUMENT'
        )['embedding']
        self.embeddings.append(embedding)
        self.documents.append(doc)

    def search(self, query, k=3):
        query_embedding = genai.embed_content(
            model='models/embedding-001',
            content=query,
            task_type='RETRIEVAL_QUERY'
        )['embedding']

        similarities = cosine_similarity([query_embedding], self.embeddings)[0] # Calculate cosine similarity between query and all document embeddings
        top_k_indices = np.argsort(similarities)[-k:][::-1] # Get indices of top k most similar documents
        return [(self.documents[i], similarities[i]) for i in top_k_indices] # Return the top k documents and their similarity scores

print("VectorStore class defined.")

VectorStore class defined.


# Customer Support Agent Implementation
# This cell defines the `CustomerSupportAgent` class, which uses the `VectorStore` to handle user queries.

In [3]:
import google.generativeai as genai

# Customer Support Agent (Day 3: Generative AI Agents)
class CustomerSupportAgent:
    def __init__(self, vector_store):
        self.vector_store = vector_store
        self.context = """
        You are a customer support agent for an e-commerce platform.
        Use professional, friendly language. Provide accurate information based on the context.
        If unsure, escalate to a human representative.
        """ # This context guides the Gemini model's responses

    def handle_query(self, user_query):
        # RAG: Search vector store for relevant documents (Day 2)
        relevant_docs = self.vector_store.search(user_query)
        context = "\n".join([doc for doc, _ in relevant_docs])

        # Few-shot prompting (Day 1: Prompt Engineering)
        prompt = f"""
        {self.context}

        Relevant information:
        {context}

        Examples:
        Q: Where is my order?
        A: Could you please provide your order number so I can check the status for you?

        Q: How do I return an item?
        A: To return an item, please visit our returns portal at [website]/returns and follow the instructions there.

        User Query: {user_query}
        Response:
        """

        response = model.generate_content(prompt)
        return response.text

print("CustomerSupportAgent class defined.")

CustomerSupportAgent class defined.


# Response Evaluation Function
# This cell defines a function to evaluate the generated responses using the Gemini model.

In [4]:
# MLOps: Basic model import google.generativeai as genai
import json

# MLOps: Basic model evaluation (Day 5: MLOps for Generative AI)
def evaluate_response(response, expected_intent):
    evaluation_prompt = f"""
    Evaluate the following response for relevance and accuracy to the intent '{expected_intent}'.
    Score from 0-10, where 10 is perfectly relevant and accurate.

    Response: {response}

    Output format:
    ```json
    {{
        "score": <int>,
        "explanation": "<string>"
    }}
    ```
    """

    eval_response = model.generate_content(evaluation_prompt)
    raw_response_text = eval_response.text.strip()
    print("Raw Evaluation Response:", raw_response_text)

    # Remove triple backticks if present
    if raw_response_text.startswith("```"):
        raw_response_text = raw_response_text[3:]
    if raw_response_text.endswith("```"):
        raw_response_text = raw_response_text[:-3]

    # Remove "json" keyword if present at the beginning (case-insensitive)
    if raw_response_text.lower().startswith("json"):
        raw_response_text = raw_response_text[4:].strip()

    raw_response_text = raw_response_text.strip() # Trim again after potential removals

    print("Processed Response:", raw_response_text) # For debugging

    try:
        return json.loads(raw_response_text) # Parse the evaluation response as JSON
    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}")
        print("Problematic Response:", raw_response_text)
        return {"score": 0, "explanation": "Error: Could not parse evaluation response."}

print("evaluate_response function defined.")

evaluate_response function defined.


# Response Evaluation Function
# This cell defines a function to evaluate the generated responses using the Gemini model.

In [5]:
import google.generativeai as genai
import json

# Main execution
def main():
    # Initialize vector store
    vector_store = VectorStore()

    # Add sample documents (simulating a knowledge base)
    documents = [
        "Shipping typically takes 3-5 business days for standard delivery.",
        "Returns can be initiated within 30 days of delivery through our returns portal.",
        "For order status, customers can check their account dashboard or contact support with their order number."
    ]

    for doc in documents:
        vector_store.add_document(doc)

    # Initialize agent
    agent = CustomerSupportAgent(vector_store)

    # Example queries
    test_queries = [
        ("Where is my package?", "check order status"),
        ("Can I return my shoes?", "process return"),
        ("How long does shipping take?", "shipping information")
    ]

    # Process queries and evaluate
    results = []
    for query, intent in test_queries:
        response = agent.handle_query(query)
        evaluation = evaluate_response(response, intent)
        results.append({
            "query": query,
            "response": response,
            "evaluation": evaluation
        })

    # Display results
    for result in results:
        print(f"\nQuery: {result['query']}")
        print(f"Response: {result['response']}")
        print(f"Evaluation: Score = {result['evaluation']['score']}, "
              f"Explanation = {result['evaluation']['explanation']}")

if __name__ == "__main__":
    main()

Raw Evaluation Response: ```json
{
  "score": 8,
  "explanation": "The response is relevant to checking order status as it provides two viable options for doing so: providing the order number or checking the account dashboard.  It doesn't directly provide the status, which is why it's not a 10, but it correctly guides the user towards obtaining it. It's slightly less efficient than immediately looking up the order if the number was readily available in context, but it remains highly relevant to the user's intent."
}
```
Processed Response: {
  "score": 8,
  "explanation": "The response is relevant to checking order status as it provides two viable options for doing so: providing the order number or checking the account dashboard.  It doesn't directly provide the status, which is why it's not a 10, but it correctly guides the user towards obtaining it. It's slightly less efficient than immediately looking up the order if the number was readily available in context, but it remains highly